In [1]:
import itertools
import os
import pandas as pd
from enum import Enum
import numpy as np
import plotly.express as px
import math

In [2]:
class TableFields(Enum):
    Rule = "Rule"
    EntropyMove = "EntropyMove"
    CurrentEntropy = "CurrentEntropy"
    EntropySize = "EntropySize"
    DataType = "DataType"
    DescriptionData = "DescriptionData"
    MessageSize = "MessageSize"
    AlphabetSize = "AlphabetSize"
    GrammaticSize = "GrammaticSize"
    CalcTime = "CalcTime"


In [6]:
datasets = ['dna', 'english', 'proteins', 'sources']
pick_size = [10000]
limit_steps = [15, None]
is_largest = [True, False]
optimized = ["bruto"]
filename = "steps.csv"

In [7]:
loads = list(itertools.product(datasets, pick_size, limit_steps, is_largest, optimized))

In [8]:
loads

[('dna', 10000, 15, True, 'bruto'),
 ('dna', 10000, 15, False, 'bruto'),
 ('dna', 10000, None, True, 'bruto'),
 ('dna', 10000, None, False, 'bruto'),
 ('english', 10000, 15, True, 'bruto'),
 ('english', 10000, 15, False, 'bruto'),
 ('english', 10000, None, True, 'bruto'),
 ('english', 10000, None, False, 'bruto'),
 ('proteins', 10000, 15, True, 'bruto'),
 ('proteins', 10000, 15, False, 'bruto'),
 ('proteins', 10000, None, True, 'bruto'),
 ('proteins', 10000, None, False, 'bruto'),
 ('sources', 10000, 15, True, 'bruto'),
 ('sources', 10000, 15, False, 'bruto'),
 ('sources', 10000, None, True, 'bruto'),
 ('sources', 10000, None, False, 'bruto')]

In [11]:
def load_experiment_results(loads):
    res = {}
    counter = 0
    for directory, pick_size, limit_steps, is_largest, optimized in loads:
        try:
            arguments = [directory, pick_size, limit_steps, is_largest, optimized, filename]
            arguments = [str(arg) for arg in arguments]
            path = os.path.sep.join(arguments)
            data = pd.read_csv(path, sep=',')
            res[path] = data
            counter += 1
        except:
            print("Error")
    return res

In [12]:
res = load_experiment_results(loads)

Error
Error
Error
Error


In [13]:
current_keys = list(res.keys())
current_keys

['dna\\10000\\15\\True\\bruto\\steps.csv',
 'dna\\10000\\15\\False\\bruto\\steps.csv',
 'dna\\10000\\None\\True\\bruto\\steps.csv',
 'english\\10000\\15\\True\\bruto\\steps.csv',
 'english\\10000\\15\\False\\bruto\\steps.csv',
 'english\\10000\\None\\True\\bruto\\steps.csv',
 'proteins\\10000\\15\\True\\bruto\\steps.csv',
 'proteins\\10000\\15\\False\\bruto\\steps.csv',
 'proteins\\10000\\None\\True\\bruto\\steps.csv',
 'sources\\10000\\15\\True\\bruto\\steps.csv',
 'sources\\10000\\15\\False\\bruto\\steps.csv',
 'sources\\10000\\None\\True\\bruto\\steps.csv']

In [62]:
class NewTable(Enum):
    StartedEntropy =  "StartedEntropy"
    EndedEntropy = "EndedEntropy"
    DiffEntropy = "DiffEntropy"
    
    StartedSizeEntropy =  "StartedSizeEntropy"
    EndedSizeEntropy = "EndedSizeEntropy"
    DiffSizeEntropy = "DiffSizeEntropy"
    
    CalculationTime = "CalculationTime"

    StartedMessageSize = "StartedMessageSize"
    EndedMessageSize = "EndedMessageSize"
    DiffMessageSize = "DiffMessageSize"

    LimitSteps = "LimitSteps"
    PickMethod = "PickMethod"
    NormalizationValue = "NormalizationValue"

    NumberOfSteps = "NumberOfSteps"

    Optimized = "Optimized"

In [63]:
def process_table(table, arguments=None):
    res = {}

    if arguments is not None:
        directory, pick_size, limit_steps, is_largest, optimized, filename = arguments
        res[NewTable.LimitSteps.value] = limit_steps 
        res[NewTable.PickMethod.value] = "Largest" if is_largest else "Random"
        res[NewTable.NormalizationValue.value] = pick_size 
        res[NewTable.Optimized.value] = optimized

    res[NewTable.StartedEntropy.value] = table.iloc[0][TableFields.CurrentEntropy.value]
    res[NewTable.EndedEntropy.value] = table.iloc[len(table) - 1][TableFields.CurrentEntropy.value]
    res[NewTable.DiffEntropy.value] = res[NewTable.EndedEntropy.value] - res[NewTable.StartedEntropy.value]

    res[NewTable.StartedSizeEntropy.value] = table.iloc[0][TableFields.EntropySize.value]
    res[NewTable.EndedSizeEntropy.value] = table.iloc[len(table) - 1][TableFields.EntropySize.value]
    res[NewTable.DiffSizeEntropy.value] = res[NewTable.EndedSizeEntropy.value] - res[NewTable.StartedSizeEntropy.value]

    res[NewTable.StartedMessageSize.value] = table.iloc[0][TableFields.MessageSize.value]
    res[NewTable.EndedMessageSize.value] = table.iloc[len(table) - 1][TableFields.MessageSize.value]
    res[NewTable.DiffMessageSize.value] = res[NewTable.StartedMessageSize.value] - res[NewTable.EndedMessageSize.value]

    res[NewTable.CalculationTime.value] = np.sum(table[TableFields.CalcTime.value].values)

    res[NewTable.NumberOfSteps.value] = len(table)

    return res

In [64]:
def process_loads(res):
    res = {}
    for directory, pick_size, limit_steps, is_largest, optimized in loads:
        try:
            arguments_raw = [directory, pick_size, limit_steps, is_largest, optimized, filename]
            arguments_str = [str(arg) for arg in arguments_raw]
            path = os.path.sep.join(arguments_str)
            data = pd.read_csv(path, sep=',')
            res[path] = process_table(data ,arguments_raw)
        except:
            print("Missing!")
    return res

In [91]:
results = pd.DataFrame.from_dict(process_loads(loads), orient="index")

Missing!
Missing!
Missing!
Missing!


In [92]:
results

,LimitSteps,PickMethod,NormalizationValue,Optimized,StartedEntropy,EndedEntropy,DiffEntropy,StartedSizeEntropy,EndedSizeEntropy,DiffSizeEntropy,StartedMessageSize,EndedMessageSize,DiffMessageSize,CalculationTime,NumberOfSteps
dna\10000\15\True\bruto\steps.csv,15.0,Largest,10000,bruto,1.979908,3.835928,1.856019,19799.081694,23851.797223,4052.715529,10000,6218,3782,37.995292,15
dna\10000\15\False\bruto\steps.csv,15.0,Random,10000,bruto,1.981966,2.852455,0.870489,19819.657628,22106.523997,2286.866369,10000,7750,2250,32.605225,15
dna\10000\None\True\bruto\steps.csv,NaN,Largest,10000,bruto,1.989577,6.007436,4.017859,19895.772726,25783.915322,5888.142596,10000,4292,5708,3126.732789,228
english\10000\15\True\bruto\steps.csv,15.0,Largest,10000,bruto,4.509031,5.156275,0.647244,45090.307640,46236.316575,1146.008935,10000,8967,1033,564.606717,15
english\10000\15\False\bruto\steps.csv,15.0,Random,10000,bruto,4.530381,4.584766,0.054385,45303.812800,45421.276767,117.463967,10000,9907,93,489.965724,15
english\10000\None\True\bruto\steps.csv,NaN,Largest,10000,bruto,4.512730,6.661476,2.148746,45127.300249,48615.449340,3488.149091,10000,7298,2702,6035.128097,273
proteins\10000\15\True\bruto\steps.csv,15.0,Largest,10000,bruto,4.202970,4.603190,0.400220,42029.704002,42920.146612,890.442610,10000,9324,676,256.088325,15
proteins\10000\15\False\bruto\steps.csv,15.0,Random,10000,bruto,4.195328,4.353783,0.158455,41953.284477,42314.418041,361.133565,10000,9719,281,227.854186,15
proteins\10000\None\True\bruto\steps.csv,NaN,Largest,10000,bruto,4.146463,7.269899,3.123436,41464.634868,46054.811671,4590.176802,10000,6335,3665,9675.256349,367
sources\10000\15\True\bruto\steps.csv,15.0,Largest,10000,bruto,5.551879,5.921528,0.369649,55518.786326,56147.928961,629.142635,10000,9482,518,1683.338367,15


- Vizualizace času
    - Porovnání jednotlivých souborů
    - Srovnání proti optimalizované metodě

- Srovnání random/largest
    - Po n krocích jak se změnilo
    - Tabulka hlavní proměnné

- Pozorování pohybu po určitém počtu kroků (size, entropy)
    - Srovnání napříč více souborů

- Finální tabulka výsledků
    - Komplet všechny důležité výsledky a jejich srovnání
    - Napříč
    - Nejlépe hlavní 

- Vizualizace času
    - Porovnání jednotlivých souborů
    - Srovnání proti optimalizované metodě

In [93]:
largest_only = results[results.PickMethod == 'Largest']
largest_only

,LimitSteps,PickMethod,NormalizationValue,Optimized,StartedEntropy,EndedEntropy,DiffEntropy,StartedSizeEntropy,EndedSizeEntropy,DiffSizeEntropy,StartedMessageSize,EndedMessageSize,DiffMessageSize,CalculationTime,NumberOfSteps
dna\10000\15\True\bruto\steps.csv,15.0,Largest,10000,bruto,1.979908,3.835928,1.856019,19799.081694,23851.797223,4052.715529,10000,6218,3782,37.995292,15
dna\10000\None\True\bruto\steps.csv,NaN,Largest,10000,bruto,1.989577,6.007436,4.017859,19895.772726,25783.915322,5888.142596,10000,4292,5708,3126.732789,228
english\10000\15\True\bruto\steps.csv,15.0,Largest,10000,bruto,4.509031,5.156275,0.647244,45090.307640,46236.316575,1146.008935,10000,8967,1033,564.606717,15
english\10000\None\True\bruto\steps.csv,NaN,Largest,10000,bruto,4.512730,6.661476,2.148746,45127.300249,48615.449340,3488.149091,10000,7298,2702,6035.128097,273
proteins\10000\15\True\bruto\steps.csv,15.0,Largest,10000,bruto,4.202970,4.603190,0.400220,42029.704002,42920.146612,890.442610,10000,9324,676,256.088325,15
proteins\10000\None\True\bruto\steps.csv,NaN,Largest,10000,bruto,4.146463,7.269899,3.123436,41464.634868,46054.811671,4590.176802,10000,6335,3665,9675.256349,367
sources\10000\15\True\bruto\steps.csv,15.0,Largest,10000,bruto,5.551879,5.921528,0.369649,55518.786326,56147.928961,629.142635,10000,9482,518,1683.338367,15
sources\10000\None\True\bruto\steps.csv,NaN,Largest,10000,bruto,5.457019,6.693041,1.236022,54570.188721,56542.809159,1972.620438,10000,8448,1552,5123.487468,226


In [73]:
fig = px.bar(
    largest_only,
    title="Čas zpracování",
    x=['Dna', 'English', 'Proteins', 'Sources'],
    y='CalculationTime', 
    text=[("%.2f" % x) for x in largest_only.CalculationTime.values],
    labels=dict(x="Typ", CalculationTime="Doba zpracování")
)

fig.show()

In [74]:
fig = px.bar(
    largest_only,
    title="Čas zpracování",
    x=['Dna', 'English', 'Proteins', 'Sources'],
    y='CalculationTime', 
    text=[("%.2f" % x) for x in largest_only.CalculationTime.values],
    labels=dict(x="Typ", CalculationTime="Doba zpracování")
)

fig.show()

- Srovnání random/largest
    - Po n krocích jak se změnilo
    - Tabulka hlavní proměnné

In [75]:
def create_df_largest_random(input_df, started_column, ended_column):
    largest_only = input_df[input_df.PickMethod == 'Largest']
    random_only = input_df[input_df.PickMethod == 'Random']

    started = largest_only[started_column].values

    ended_largest = largest_only[ended_column].values
    ended_random = random_only[ended_column].values



    ended_df_largest = pd.DataFrame()
    ended_df_largest["Value"] = ended_largest
    ended_df_largest['Type'] = 'Ended_Largest'
    ended_df_largest['Path'] = largest_only.index

    started_df = pd.DataFrame()
    started_df["Value"] = started
    started_df['Type'] = 'Started'
    started_df['Path'] = largest_only.index


    ended_df_random = pd.DataFrame()
    ended_df_random["Value"] = ended_random
    ended_df_random['Type'] = 'Ended_Random'
    ended_df_random['Path'] = largest_only.index


    res = pd.concat([started_df, ended_df_largest, ended_df_random])

    return res

In [76]:
msg_size = create_df_largest_random(results, NewTable.StartedMessageSize.value, NewTable.EndedMessageSize.value)

fig = px.bar(
    msg_size, 
    x=['Dna', 'English', 'Proteins', 'Sources']*3, 
    y="Value",
    color='Type', 
    barmode='group',
    text=[("%.2f" % x) for x in msg_size.Value.values],
    title="Změna velikost zprávy po 15 krocích",
    labels=dict(x="Typ", Value="Velikost zprávy")
)

fig.show()

KeyError: (nan, '', '', '', '')

In [77]:
entropy = create_df_largest_random(results, NewTable.StartedSizeEntropy.value, NewTable.EndedSizeEntropy.value)

fig = px.bar(
    entropy, 
    x=['Dna', 'English', 'Proteins', 'Sources']*3, 
    y="Value",
    color='Type', 
    barmode='group',
    text=[("%.2f" % x) for x in entropy.Value.values],
    title="Změna entropie zprávy po 15 krocích",
    labels=dict(x="Typ", Value="Entropie zprávy")
)

fig.show()


KeyError: (nan, '', '', '', '')

- Pozorování pohybu po určitém počtu kroků (size, entropy)
    - Srovnání napříč více souborů

In [104]:
def create_df(input_df, started_column, ended_column):
    largest_only = input_df[input_df.PickMethod == 'Largest']

    index = largest_only[largest_only.NumberOfSteps == 15].index

    started = largest_only[largest_only.NumberOfSteps == 15]
    started = started[started_column].values

    ended_15 = largest_only[largest_only.NumberOfSteps == 15]
    ended_15 = ended_15[ended_column].values

    ended_None = largest_only[largest_only.NumberOfSteps != 15]
    ended_None = ended_None[ended_column].values

    started_df = pd.DataFrame()
    started_df["Value"] = started
    started_df['Type'] = 'Started'
    started_df['Path'] = index

    ended_df = pd.DataFrame()
    ended_df["Value"] = ended_15
    ended_df['Type'] = 'Ended 15'
    ended_df['Path'] = index


    ended_df_none = pd.DataFrame()
    ended_df_none["Value"] = ended_None
    ended_df_none['Type'] = 'Ended All'
    ended_df_none['Path'] = index

    res = pd.concat([started_df, ended_df, ended_df_none])

    return res

In [107]:
msg_size = create_df(results, NewTable.StartedMessageSize.value, NewTable.EndedMessageSize.value)
msg_size

,Value,Type,Path
0,10000,Started,dna\10000\15\True\bruto\steps.csv
1,10000,Started,english\10000\15\True\bruto\steps.csv
2,10000,Started,proteins\10000\15\True\bruto\steps.csv
3,10000,Started,sources\10000\15\True\bruto\steps.csv
0,6218,Ended 15,dna\10000\15\True\bruto\steps.csv
1,8967,Ended 15,english\10000\15\True\bruto\steps.csv
2,9324,Ended 15,proteins\10000\15\True\bruto\steps.csv
3,9482,Ended 15,sources\10000\15\True\bruto\steps.csv
0,4292,Ended All,dna\10000\15\True\bruto\steps.csv
1,7298,Ended All,english\10000\15\True\bruto\steps.csv


In [108]:
fig = px.bar(
    msg_size, 
    x=['Dna', 'English', 'Proteins', 'Sources']*3, 
    y="Value",
    color='Type', 
    barmode='group',
    text=[("%.2f" % x) for x in msg_size.Value.values],
    title="Změna velikost zprávy po 15 krocích",
    labels=dict(x="Typ", Value="Velikost zprávy")
)

fig.show()

In [109]:
entropy = create_df(results, NewTable.StartedSizeEntropy.value, NewTable.EndedSizeEntropy.value)
entropy

,Value,Type,Path
0,19799.081694,Started,dna\10000\15\True\bruto\steps.csv
1,45090.307640,Started,english\10000\15\True\bruto\steps.csv
2,42029.704002,Started,proteins\10000\15\True\bruto\steps.csv
3,55518.786326,Started,sources\10000\15\True\bruto\steps.csv
0,23851.797223,Ended 15,dna\10000\15\True\bruto\steps.csv
1,46236.316575,Ended 15,english\10000\15\True\bruto\steps.csv
2,42920.146612,Ended 15,proteins\10000\15\True\bruto\steps.csv
3,56147.928961,Ended 15,sources\10000\15\True\bruto\steps.csv
0,25783.915322,Ended All,dna\10000\15\True\bruto\steps.csv
1,48615.449340,Ended All,english\10000\15\True\bruto\steps.csv


In [111]:
fig = px.bar(
    entropy, 
    x=['Dna', 'English', 'Proteins', 'Sources']*3, 
    y="Value",
    color='Type', 
    barmode='group',
    text=[("%.2f" % x) for x in entropy.Value.values],
    title="Změna entropie zprávy po 15 krocích",
    labels=dict(x="Typ", Value="Entropie zprávy")
)

fig.show()


In [81]:
msg_size

,Value,Type,Path
0,10000,Started,dna\10000\None\True\bruto\steps.csv
1,10000,Started,english\10000\None\True\bruto\steps.csv
2,10000,Started,proteins\10000\None\True\bruto\steps.csv
3,10000,Started,sources\10000\None\True\bruto\steps.csv
0,4292,Ended,dna\10000\None\True\bruto\steps.csv
1,7298,Ended,english\10000\None\True\bruto\steps.csv
2,6335,Ended,proteins\10000\None\True\bruto\steps.csv
3,8448,Ended,sources\10000\None\True\bruto\steps.csv


In [113]:
df_steps = results[results.LimitSteps != 15]

In [114]:
df_steps

,LimitSteps,PickMethod,NormalizationValue,Optimized,StartedEntropy,EndedEntropy,DiffEntropy,StartedSizeEntropy,EndedSizeEntropy,DiffSizeEntropy,StartedMessageSize,EndedMessageSize,DiffMessageSize,CalculationTime,NumberOfSteps
dna\10000\None\True\bruto\steps.csv,NaN,Largest,10000,bruto,1.989577,6.007436,4.017859,19895.772726,25783.915322,5888.142596,10000,4292,5708,3126.732789,228
english\10000\None\True\bruto\steps.csv,NaN,Largest,10000,bruto,4.512730,6.661476,2.148746,45127.300249,48615.449340,3488.149091,10000,7298,2702,6035.128097,273
proteins\10000\None\True\bruto\steps.csv,NaN,Largest,10000,bruto,4.146463,7.269899,3.123436,41464.634868,46054.811671,4590.176802,10000,6335,3665,9675.256349,367
sources\10000\None\True\bruto\steps.csv,NaN,Largest,10000,bruto,5.457019,6.693041,1.236022,54570.188721,56542.809159,1972.620438,10000,8448,1552,5123.487468,226


In [120]:
fig = px.bar(
    df_steps, 
    x=['Dna', 'English', 'Proteins', 'Sources']*1, 
    y="NumberOfSteps",
    color=df_steps.index, 
    barmode='group',
    text=[("%.2f" % x) for x in df_steps.NumberOfSteps.values],
    title="Počet kroků algoritmu",
    labels=dict(x="Typ", NumberOfSteps="Počet provedených iterací")
)

fig.show()


- Finální tabulka výsledků
    - Komplet všechny důležité výsledky a jejich srovnání
    - Napříč
    - Nejlépe hlavní 

In [82]:
results

,LimitSteps,PickMethod,NormalizationValue,Optimized,StartedEntropy,EndedEntropy,DiffEntropy,StartedSizeEntropy,EndedSizeEntropy,DiffSizeEntropy,StartedMessageSize,EndedMessageSize,DiffMessageSize,CalculationTime,NumberOfSteps
dna\10000\None\True\bruto\steps.csv,NaN,Largest,10000,bruto,1.989577,6.007436,4.017859,19895.772726,25783.915322,5888.142596,10000,4292,5708,3126.732789,228
english\10000\None\True\bruto\steps.csv,NaN,Largest,10000,bruto,4.512730,6.661476,2.148746,45127.300249,48615.449340,3488.149091,10000,7298,2702,6035.128097,273
proteins\10000\None\True\bruto\steps.csv,NaN,Largest,10000,bruto,4.146463,7.269899,3.123436,41464.634868,46054.811671,4590.176802,10000,6335,3665,9675.256349,367
sources\10000\None\True\bruto\steps.csv,NaN,Largest,10000,bruto,5.457019,6.693041,1.236022,54570.188721,56542.809159,1972.620438,10000,8448,1552,5123.487468,226


In [83]:
def create_final_df(input_df=results):
    final_results = input_df.copy()

    final_results['MessageSize Diff %'] = [(100 - (100 * (end/start))) for start, end in zip(final_results.StartedMessageSize.values, final_results.EndedMessageSize.values)]

    final_results['Entropy Diff %'] = [round(abs(((100 * (start/end)) - 100)), 2) for start, end in zip(final_results.StartedEntropy.values, final_results.EndedEntropy.values)]

    final_results['Type'] = list(map(lambda x: x.split(os.path.sep)[0], final_results.index)) 

    final_results = final_results.drop([
        NewTable.EndedEntropy.value,
        NewTable.StartedEntropy.value,
        NewTable.StartedMessageSize.value,
        NewTable.EndedMessageSize.value,
        NewTable.StartedSizeEntropy.value,
        NewTable.EndedSizeEntropy.value,
        NewTable.NormalizationValue.value,
        NewTable.Optimized.value,
    ], axis=1)

    final_results = final_results.set_index("Type")

    return final_results

In [84]:
f_res = create_final_df()

In [85]:
f_res

,LimitSteps,PickMethod,DiffEntropy,DiffSizeEntropy,DiffMessageSize,CalculationTime,NumberOfSteps,MessageSize Diff %,Entropy Diff %
Type,,,,,,,,,
dna,NaN,Largest,4.017859,5888.142596,5708,3126.732789,228,57.08,66.88
english,NaN,Largest,2.148746,3488.149091,2702,6035.128097,273,27.02,32.26
proteins,NaN,Largest,3.123436,4590.176802,3665,9675.256349,367,36.65,42.96
sources,NaN,Largest,1.236022,1972.620438,1552,5123.487468,226,15.52,18.47


# Grafové srovnání metod

In [52]:
list(res.keys())

['dna\\10000\\15\\True\\bruto\\steps.csv',
 'dna\\10000\\15\\False\\bruto\\steps.csv',
 'dna\\10000\\None\\True\\bruto\\steps.csv',
 'english\\10000\\15\\True\\bruto\\steps.csv',
 'english\\10000\\15\\False\\bruto\\steps.csv',
 'english\\10000\\None\\True\\bruto\\steps.csv',
 'proteins\\10000\\15\\True\\bruto\\steps.csv',
 'proteins\\10000\\15\\False\\bruto\\steps.csv',
 'proteins\\10000\\None\\True\\bruto\\steps.csv',
 'sources\\10000\\15\\True\\bruto\\steps.csv',
 'sources\\10000\\15\\False\\bruto\\steps.csv',
 'sources\\10000\\None\\True\\bruto\\steps.csv']

In [53]:
def create_graphs_df(input_df, current_df, steps, table_value):
    current_keys = list(input_df.keys())
    keys = list(filter(lambda x: current_df in x, current_keys))
    keys = list(filter(lambda x: str(steps) in x in x, keys))

    largest_key = list(filter(lambda x: 'True' in x, keys))[0]
    random_key = list(filter(lambda x: 'False' in x, keys))[0]
    
    
    largest = largest_key
    random = random_key

    #TableFields.EntropySize.value
    #TableFields.MessageSize.value

    value_largest = input_df[largest][table_value]

    value_random = input_df[random][table_value]


    df_largest = pd.DataFrame()
    df_largest['Value'] = value_largest
    df_largest['Type'] = 'Largest'

    df_random = pd.DataFrame()
    df_random['Value'] = value_random
    df_random['Type'] = 'Random'

    return pd.concat([df_largest, df_random])


## DNA

In [54]:
t = 'dna' 

In [55]:
df_message_size = create_graphs_df(res, t, 15, TableFields.MessageSize.value)
fig = px.line(df_message_size, y="Value", color='Type',
    title="Změna velikost zprávy po 15 krocích",
    labels=dict(index="Čas", Value="Velikost zprávy")
)
fig.show()

df_entropy = create_graphs_df(res, t, 15, TableFields.EntropySize.value)
fig = px.line(df_entropy, y="Value", color='Type',
    title="Změna entropie zprávy po 15 krocích",
    labels=dict(index="Čas", Value="Entropie zprávy")
)
fig.show()

## Proteins

In [56]:
t = 'proteins'

In [57]:
df_message_size = create_graphs_df(res, t, 15, TableFields.MessageSize.value)
fig = px.line(df_message_size, y="Value", color='Type',
    title="Změna velikost zprávy po 15 krocích",
    labels=dict(index="Čas", Value="Velikost zprávy")
)
fig.show()

df_entropy = create_graphs_df(res, t, 15, TableFields.EntropySize.value)
fig = px.line(df_entropy, y="Value", color='Type',
    title="Změna entropie zprávy po 15 krocích",
    labels=dict(index="Čas", Value="Entropie zprávy")
)
fig.show()

## Sources

In [58]:
t = 'sources'

In [59]:
df_message_size = create_graphs_df(res, t, 15, TableFields.MessageSize.value)
fig = px.line(df_message_size, y="Value", color='Type',
    title="Změna velikost zprávy po 15 krocích",
    labels=dict(index="Čas", Value="Velikost zprávy")
)
fig.show()

df_entropy = create_graphs_df(res, t, 15, TableFields.EntropySize.value)
fig = px.line(df_entropy, y="Value", color='Type',
    title="Změna entropie zprávy po 15 krocích",
    labels=dict(index="Čas", Value="Entropie zprávy")
)
fig.show()

## English

In [60]:
t = 'english'

In [61]:
df_message_size = create_graphs_df(res, t, 15, TableFields.MessageSize.value)
fig = px.line(df_message_size, y="Value", color='Type',
    title="Změna velikost zprávy po 15 krocích",
    labels=dict(index="Čas", Value="Velikost zprávy")
)
fig.show()

df_entropy = create_graphs_df(res, t, 15, TableFields.EntropySize.value)
fig = px.line(df_entropy, y="Value", color='Type',
    title="Změna entropie zprávy po 15 krocích",
    labels=dict(index="Čas", Value="Entropie zprávy")
)
fig.show()